Installing dependencies

In [ ]:
pip install requests pandas

In [ ]:
import requests
import time
import pandas as pd

API_KEY = '###' #This is my google cloud API key, you can put yours here
LOCATION = '52.34709094162969, -1.5718042268518115' # This is a decimal GPS coordinate separated by comma for Kenilsworth
RADIUS = '5000'  #this should be in metres max is 50,000, mine is doing a 5KM radius
TYPE = 'establishment' #This is the businesses or location type I am targetting according to Google places and maps, mine is targetting all businesses
FIELDS = 'name,formatted_address,international_phone_number,website,types' #These are the data fields I would like to get from places API, you can add more fields if needed

#Writing a function to get the places in a list

def get_places():
  url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
  params = {
      'location': LOCATION,
      'radius': RADIUS,
      'type': TYPE,
      'key': API_KEY
  }

  results = []

  while True:
    response = requests.get(url, params =params)
    print(f"API Status Code: {response.status_code}") # Print status code, this is to check if we are getting response from the API
    try:
        data = response.json()
        print(f"API Response Data: {data}") # Print response data
    except requests.exceptions.JSONDecodeError:
        print(f"Could not decode JSON from response: {response.text}")
        break

    results.extend(data.get('results', []))

    token = data.get('next_page_token')
    if token:
      time.sleep(2) #Google short delay
      params = {'pagetoken':token, 'key': API_KEY}
    else:
      break
  print(f"Found {len(results)} raw places.") # This helps to get the length of the requested data
  return results

  #Function to get datials of each place

def get_details(place_id):
    url = 'https://maps.googleapis.com/maps/api/place/details/json'
    params = {
        'place_id': place_id,
        'fields': FIELDS,
        'key': API_KEY
    }
    response = requests.get(url, params=params)
    return response.json().get('result', {})

#function to extract

def extract_info(raw_places):
    data = []
    for place in raw_places:
        place_id = place.get('place_id')
        details = get_details(place_id)
        time.sleep(0.2)  # Avoid hitting rate limits
        if details: # Add this check
            data.append({
                'Name': details.get('name'),
                'Address': details.get('formatted_address'),
                'Phone': details.get('international_phone_number'),
                'Website': details.get('website'),
                'Types': ", ".join(details.get('types', []))
            })
    print(f"Extracted info for {len(data)} places.") # Add this print statement
    return data


# Main Execution
raw_places = get_places()
business_data = extract_info(raw_places)

# Export to CSV
df = pd.DataFrame(business_data)
df.to_csv('Kenilsworth_businesses.csv', index=False)
print("CSV exported successfully.")

API Status Code: 200
API Response Data: {'html_attributions': [], 'next_page_token': 'ATKogpeLf__STlD_mOlmrStXjnuJDGdLJnTj6RgeKvrMvtSurX1au6SgpylnOFOAC7oB4_HLh8LIun_W_NI8n-d4JK46s_Y1bifLzEcWq5asyRBaCL5Chs0tK6hK_XbDT0HoMYY6WyjGMyY_yBQOeZJsdCD_zihIANK0PY3g535lduATXNN5PKqOF2nDZxbcgU9Q1obiyG7LbvBP7yFRRhJT1w0eU5vy_2bt4LUHLBFJ2Dl4TogrY6tipvkQGr9I0vdw4ZyEekXUabw1QDC6__BVKyE0rrqj9Dzs_klEo_l-BOBEenbMsDreWDicjkidPITVJR16Xx2uhINYnHhregncVNr9B75YfFSce43zmlutov-gArthzfumNFskDfWGVXqvnevd61H165DZ-NBfjldlpa8wACgI_UhPIeW57tRo49qO', 'results': [{'geometry': {'location': {'lat': 52.4128163, 'lng': -1.5089521}, 'viewport': {'northeast': {'lat': 52.46347227234299, 'lng': -1.423950826722567}, 'southwest': {'lat': 52.36391078072344, 'lng': -1.605885287021431}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet', 'name': 'Coventry', 'photos': [{'he